## Separation Anxiety

In [ ]:
%matplotlib widget

In [ ]:
from math import *  # type: ignore


class Vec:
    def __init__(self, x: float, y: float, z: float) -> None:
        self.x = x
        self.y = y
        self.z = z

    def dot(self, other: "Vec") -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z

    @staticmethod
    def Survey(inc: float, azi: float) -> "Vec":
        sini, cosi = sin(inc), cos(inc)
        sina, cosa = sin(azi), cos(azi)
        return Vec(sini * sina, sini * cosa, cosi)

    def normalize(self) -> float:
        n = self.norm()
        if n != 0:
            s = 1 / n
            self.x *= s
            self.y *= s
            self.z *= s
        return n

    def norm(self) -> float:
        return sqrt(self.dot(self))

    def copy(self) -> "Vec":
        return Vec(self.x, self.y, self.z)

    def __add__(self, other: "Vec") -> "Vec":
        return Vec(self.x + other.x, self.y + other.y, self.z + other.z)

    def __sub__(self, other: "Vec") -> "Vec":
        return Vec(self.x - other.x, self.y - other.y, self.z - other.z)

    def __rmul__(self, s: float) -> "Vec":
        return Vec(self.x * s, self.y * s, self.z * s)

    def __str__(self) -> str:
        return f"({self.x:.2f}, {self.y:.2f}, {self.z:.2f})"

In [ ]:
from math import *  # type: ignore


class Ellipse:
    def __init__(self, a: float, b: float, r: float):
        self.a = a
        self.b = b
        self.set_rotation(r)
        self.o = Vec(0, 0, 0)

    def set_rotation(self, rot: float) -> None:
        self.rot = rot
        sinr = sin(rot)
        cosr = cos(rot)
        self.du = Vec(cosr, sinr, 0.0)
        self.dv = Vec(-sinr, cosr, 0.0)
        self.u = self.a * self.du
        self.v = self.b * self.dv

    def radius(self, t: float) -> float:
        sinr = sin(t)
        cosr = cos(t)
        p = Vec(cosr, sinr, 0)
        return 1 / sqrt(p.x * p.x / (self.a * self.a) + p.y * p.y / (self.b * self.b))

    def parameterized(self, t: float) -> Vec:
        sint = sin(t)
        cost = cos(t)
        return cost * self.u + sint * self.v

    def project(self, p: Vec) -> Vec:
        return Vec(p.dot(self.du), p.dot(self.dv), 0)

    def rotate(self, p: Vec) -> Vec:
        return p.x * self.du + p.y * self.dv

    def tangent(self, s: float) -> tuple[Vec, Vec]:
        r = self.radius(s)
        p = Vec(r * cos(s), r * sin(s), 0)
        t = Vec(p.y / (self.b * self.b), -p.x / (self.a * self.a), 0)
        return self.rotate(p), self.rotate(t)

    def scale(self, k: float) -> "Ellipse":
        scaled = Ellipse(self.a * k, self.b * k, self.rot)
        scaled.o = self.o
        return scaled

In [ ]:
class Cov:
    def __init__(self):
        self.ee = 0.0
        self.nn = 0.0
        self.vv = 0.0
        self.en = 0.0
        self.ev = 0.0
        self.nv = 0.0

    def add(self, vec: Vec):
        self.ee += vec.x * vec.x
        self.nn += vec.y * vec.y
        self.vv += vec.z * vec.z
        self.en += vec.x * vec.y
        self.ev += vec.x * vec.z
        self.nv += vec.y * vec.z

    def pedal(self, vec: Vec):
        x = vec.x * (vec.x * self.ee + vec.y * self.en + vec.z * self.ev)
        y = vec.y * (vec.x * self.en + vec.y * self.nn + vec.z * self.nv)
        z = vec.z * (vec.x * self.ev + vec.y * self.nv + vec.z * self.vv)
        return x + y + z

    def radius(self, vec: Vec):
        return sqrt(abs(self.pedal(vec)))

    def __str__(self) -> str:
        return f"[{self.ee:8.3f},{self.nn:8.3f},{self.vv:8.3f},{self.en:8.3f},{self.ev:8.3f},{self.nv:8.3f}]"


def lha(inc, azi):
    sini, cosi = sin(inc), cos(inc)
    sina, cosa = sin(azi), cos(azi)
    return (
        Vec(cosa, -sina, 0),
        Vec(cosi * sina, cosi * cosa, -sini),
        Vec(sini * sina, sini * cosa, cosi),
    )

In [ ]:
import numpy as np

def ellipse_to_cov(ell: Ellipse) -> Cov:
    cov = Cov()
    cov.add(ell.a * ell.du)
    cov.add(ell.b * ell.dv)
    return cov


def combined_ellipse_to_cov(e1: Ellipse, e2: Ellipse) -> Cov:
    cov = Cov()
    cov.add(e1.a * e1.du)
    cov.add(e1.b * e1.dv)
    cov.add(e2.a * e2.du)
    cov.add(e2.b * e2.dv)
    return cov


def cov_to_ellipse(cov: Cov) -> Ellipse:
    a = np.array([[cov.ee, cov.en], [cov.en, cov.nn]])
    svd = np.linalg.svd(a)
    u = svd.U[0]
    theta = atan2(u[1], u[0])
    return Ellipse(sqrt(svd.S[0]), sqrt(svd.S[1]), theta)

In [ ]:
from typing import Callable, Optional
from matplotlib.axes import Axes
from matplotlib.lines import Line2D
from scipy.stats.distributions import chi2
import ipywidgets as widgets
import matplotlib.pyplot as plt


class Setup:
    k = 3.5


setup = Setup()

def std_rot(rot:float)->float:
    rot = fmod(rot, 2 * pi)
    return rot if rot >- 0 else rot + 2*pi

class Scaled:
    e1: Ellipse
    e1lines: list[Line2D] | None
    e2: Ellipse
    e2lines: list[Line2D] | None
    e3: Ellipse
    e3lines: list[Line2D] | None
    e4: Ellipse
    e4lines: list[Line2D] | None
    e1scaledlines: list[Line2D] | None
    e2scaledlines: list[Line2D] | None
    observer:Optional[Callable]

    def __init__(self):
        self.sep = Vec(0, 0, 0)
        self.radius = 0.0
        self.k = 1.0
        self.cov = Cov()
        self.e1 = Ellipse(200, 100, radians(180))
        self.e1lines = None
        self.e2 = Ellipse(400, 20, radians(180))
        self.e2lines = None
        self.e1.o = Vec(400, 30, 0)
        self.e2.o = Vec(40, 180, 0)
        self.e3 = Ellipse(1, 1, 0)
        self.e3lines = None
        self.e4 = Ellipse(1, 1, 0)
        self.e4lines = None
        self.parameter = np.linspace(0, 2 * pi, 100)
        self.e1scaledlines = None
        self.e2scaledlines = None
        self.observer = None

    def compute(self):
        self.sep = sep = self.e2.o - self.e1.o
        e3 = self.e3
        lensep = self.sep.normalize()
        self.radius = radius = e3.radius(atan2(sep.dot(e3.dv), sep.dot(e3.du)))
        self.k = lensep / radius

    def make_ecombined(self) -> Ellipse:
        cov = self.cov = combined_ellipse_to_cov(self.e1, self.e2)
        self.e3 = ecomb = cov_to_ellipse(cov)
        self.e3.o = self.e1.o
        self.compute()
        return self.e3

    def _points(self, ell: Ellipse) -> tuple[list[float], list[float]]:
        x = [p.x for p in (ell.o + ell.parameterized(float(r)) for r in self.parameter)]
        y = [p.y for p in (ell.o + ell.parameterized(float(r)) for r in self.parameter)]
        return x, y

    def plot4(self):
        self.e4 = self.e3.scale(self.k)
        x, y = self._points(self.e4)
        if self.e4lines is not None:
            self.e4lines[0].set_data(x, y)
        else:
            self.e4lines = ax.plot(x, y, "grey")

    def plot3(self):
        e3_scaled = self.e3.scale(setup.k)
        x, y = self._points(e3_scaled)
        if self.e3lines is not None:
            self.e3lines[0].set_data(x, y)
        else:
            self.e3lines = ax.plot(x, y, "#fedada")

    def plot1(self, rot: float, solo:bool=False):
        self.e1.set_rotation(std_rot(rot))
        self.make_ecombined()
        self.plot1_scaled(setup.k)
        x, y = self._points(self.e1)
        if self.e1lines is not None:
            self.e1lines[0].set_data(x, y)
            if not solo:
                self.plot3()
                self.plot4()
        else:
            self.e1lines = ax.plot(x, y)
        if not solo:
            self.notify()

    def plot1_scaled(self, k: float):
        e1_scaled = self.e1.scale(k)
        x, y = self._points(e1_scaled)
        if self.e1scaledlines is not None:
            self.e1scaledlines[0].set_data(x, y)
        else:
            self.e1scaledlines = ax.plot(x, y, "#ececec")

    def plot2(self, rot: float):
        self.e2.set_rotation(std_rot(rot))
        self.make_ecombined()
        self.plot2_scaled(setup.k)
        x, y = self._points(self.e2)
        if self.e2lines is not None:
            self.e2lines[0].set_data(x, y)
            self.plot3()
            self.plot4()
        else:
            self.e2lines = ax.plot(x, y)
        self.notify()

    def notify(self):
        if self.observer is not None:
            self.observer(self)

    def plot2_scaled(self, k: float):
        e2_scaled = self.e2.scale(k)
        x, y = self._points(e2_scaled)
        if self.e2scaledlines is not None:
            self.e2scaledlines[0].set_data(x, y)
        else:
            self.e2scaledlines = ax.plot(x, y, color="#ececec")

    def plot(self, ax: Axes):
        ax.set_xlim(*xlim(self.e1, self.e2))
        ax.set_ylim(*ylim(self.e1, self.e2))
        ax.set_aspect("equal")

        self.make_ecombined()
        # Combined ellipse
        self.plot3()

        # Scaled up ellipses
        self.plot1_scaled(setup.k)
        self.plot2_scaled(setup.k)
        # Combined, scaled ellipse
        self.plot4()

        # 1-sigma ellipses
        self.plot1(self.e1.rot, solo=True)
        self.plot2(self.e2.rot)

        # Center line
        ax.plot([self.e1.o.x, self.e2.o.x], [self.e1.o.y, self.e2.o.y])
        ax.text(self.e1.o.x, self.e1.o.y, "B")
        ax.text(self.e2.o.x, self.e2.o.y, "A")

    def perp(self):
        self.parallel(atan2(-self.sep.y, -self.sep.x) + pi / 2)

    def parallel(self, angle):
        self.plot1(angle, solo=True)
        self.plot2(angle)

    def rotate(self, angle):
        self.plot1(self.e1.rot + angle, solo=True)
        self.plot2(self.e2.rot + angle)


class Tally:
    xmin = 1e12
    xmax = -1e12

    def tally(self, x):
        if x < self.xmin:
            self.xmin = x
        if x > self.xmax:
            self.xmax = x


def drss(*ell: Ellipse) -> float:
    s = 0.0
    for e in ell:
        d = max(e.a, e.b)
        s += d * d
    s = sqrt(s) * 1.1
    return s


def xlim(*ell: Ellipse) -> tuple[float, float]:
    t = Tally()
    d = drss(*ell)
    for e in ell:
        t.tally(e.o.x - d)
        t.tally(e.o.x + d)
    return t.xmin, t.xmax


def ylim(*ell: Ellipse) -> tuple[float, float]:
    t = Tally()
    d = drss(*ell)
    for e in ell:
        t.tally(e.o.y - d)
        t.tally(e.o.y + d)
    return t.xmin, t.xmax

In [ ]:

plt.close()
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(10, 8)
fig.set_layout_engine("tight")

ax.set_aspect("equal")

scaled = Scaled()
ra = widgets.IntSlider(
    description="A rot", min=0, max=360, value=degrees(scaled.e2.rot)
)
rb = widgets.IntSlider(
    description="B rot", min=0, max=360, value=degrees(scaled.e1.rot)
)
def update(s: Scaled):
    global updating
    try:
        updating = True
        ra.value = degrees(s.e2.rot)
        rb.value = degrees(s.e1.rot)
    finally:
        updating = False

scaled.observer = update
scaled.plot(ax)

updating = False


def on_rota(r):
    if not updating:
        scaled.plot2(radians(r.new))


def on_rotb(r):
    if not updating:
        scaled.plot1(radians(r.new))


ra.observe(on_rota, names="value")
rb.observe(on_rotb, names="value")



def showconf():
    # Separation factor
    sep = scaled.e2.o - scaled.e1.o
    c2c = sep.normalize()
    pua = ellipse_to_cov(scaled.e1)
    apcr = pua.radius(sep)
    pub = ellipse_to_cov(scaled.e2)
    bpcr = pub.radius(sep)
    sf = c2c / (setup.k * hypot(apcr, bpcr))
    # P value
    p = sqrt(chi2.cdf(scaled.k * scaled.k, 3))
    conf_label.value = f"P-value: {p:.4f} Simple SF: {sf:.3f}"
    sf_values.value = (
        f"Apcr={apcr:.1f} Bpcr={bpcr:.1f} K={setup.k:.2f} rss={hypot(apcr, bpcr):.1f}"
    )
    pass


def set_conf_scale(v: float):
    setup.k = v
    scaled.plot1_scaled(setup.k)
    scaled.plot2_scaled(setup.k)
    scaled.plot3()
    showconf()

H = widgets.HBox
V = widgets.VBox

conf_label = widgets.Label(value="??%")
sf_values = widgets.Label(value="---")
conf_scale = widgets.FloatSlider(
    description="K", value=setup.k, min=1.0, max=4.0, step=0.01
)
halfwidth = widgets.Layout(width='50%')
button_align = widgets.Button(description="Align")
button_align.on_click(lambda b: scaled.perp())
button_90 = widgets.Button(description="90", layout=halfwidth)
button_90.on_click(lambda b: scaled.parallel(pi / 2))
button_0 = widgets.Button(description="0", layout=halfwidth)
button_0.on_click(lambda b: scaled.parallel(pi))
button_more = widgets.Button(description="+30", layout=halfwidth)
button_more.on_click(lambda b:scaled.rotate(pi/12))
button_less = widgets.Button(description="-30", layout=halfwidth)
button_less.on_click(lambda b:scaled.rotate(-pi/12))
button_bmore = widgets.Button(description="B +30", layout=halfwidth)
button_bmore.on_click(lambda b:scaled.plot1(scaled.e1.rot + pi/12))
button_bless = widgets.Button(description="B -30", layout=halfwidth)
button_bless.on_click(lambda b:scaled.plot1(scaled.e1.rot - pi/12))
button_amore = widgets.Button(description="A +30", layout=halfwidth)
button_amore.on_click(lambda b:scaled.plot2(scaled.e2.rot + pi/12))
button_aless = widgets.Button(description="A -30", layout=halfwidth)
button_aless.on_click(lambda b:scaled.plot2(scaled.e2.rot - pi/12))

ra.observe(lambda v: showconf(), names="value")
rb.observe(lambda v: showconf(), names="value")
conf_scale.observe(lambda v: set_conf_scale(v.new), names="value")

showconf()
display(
    widgets.HBox(
        [
            V([button_align, H([button_90, button_0]), H([button_more,button_less]),]),
            V([ra, conf_scale, H([button_amore, button_aless])]),
            V([rb, widgets.Label(), H([button_bmore, button_bless])]),
            V([conf_label, sf_values, ]),
        ]
    )
)
plt.show()

Simple SF ACR=`Separation / (K * Sqrt(Apcr^2 + Bpcr^2))`

Key
- Blue ellipse "1&sigma;" at B
- Orange ellipse "1&sigma;" at A
- Pale grey ellipses at A and B are scaled by K
- Pink ellipse is A combined (randomly) with B scaled by K - its pedal radius is used in the separation factor as `K * Sqrt(Apcr^2 + Bpcr^2)`

Dark grey ellipse is combined ellipse scaled to intersect A
It is the probability "contour" on which A lies
